In [1]:
%%bash
echo "deb [ arch=amd64 ] https://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
apt-get update
apt-get install python3-graph-tool

Executing: /tmp/apt-key-gpghome.zzYpwt6MM3/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:8 https://downloads.skewed.de/apt bionic InRelease [7,540 B]
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get

gpg: key 612DEFB798507F25: public key "Tiago de Paula Peixoto <tiago@skewed.de>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
import pandas as pd
edges = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_30last/edges.parquet')
edges_test = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_30last/edges_test.parquet')

In [2]:
#splitted
import pandas as pd
edges = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_30last/edges.parquet')
edges_test = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_30last/edges_test.parquet')

In [1325]:
#Params
TEST_CLICK_WEIGHT = 3
TEST_CART_WEIGHT = 6
TEST_ORDER_WEIGHT_L= 6
TEST_ORDER_WEIGHT_S = 2


CLICK_WEIGHT = 3
CART_WEIGHT = 10
ORDER_WEIGHT = 1

TOPK_COUSER = 100
ALPHA = 0.5
BETA = 0.5

# ORDER_FILTER = 0.7

RERANK_L = 1
RERANK_R = 1

USE_PAGERANK = False
USE_LOG = False

# Create graph

In [4]:
from graph_tool import Graph
ug = Graph(directed=False)

In [5]:
edges.iloc[:, 1] = edges.iloc[:,1]  + 100000000
edges.shape

(94282512, 7)

In [6]:
edges

,"('session', '')","('aid', '')","('ts', 'last')","('ts', 'first')","('clicks', 'sum')","('carts', 'sum')","('orders', 'sum')"
0,0,100016246,62919,62919,1,0,0
1,0,100030373,1798887,1798887,1,0,0
2,0,100097836,1590509,1590509,1,0,0
3,0,100102416,1714839,1714839,1,0,0
4,0,100154930,1241409,1241409,1,0,0
...,...,...,...,...,...,...,...
94282507,11098523,100175715,1814397,1814397,1,0,0
94282508,11098524,101088524,1814398,1814398,1,0,0
94282509,11098525,100182927,1814399,1814399,1,0,0
94282510,11098526,100510055,1814399,1814399,1,0,0


In [7]:
edges_test.iloc[:, 1]= edges_test.iloc[:, 1] + 100000000
edges_test.shape

(5535990, 7)

In [8]:
edges_test

,"('session', '')","('aid', '')","('ts', 'last')","('ts', 'first')","('clicks', 'sum')","('carts', 'sum')","('orders', 'sum')"
0,11098528,100011830,1814400,1814400,1,0,0
1,11098529,101105029,1814400,1814400,1,0,0
2,11098530,100264500,1814488,1814400,2,0,0
3,11098530,100409236,1815732,1814569,3,1,0
4,11098531,100396199,1814946,1814468,2,0,1
...,...,...,...,...,...,...,...
5535985,12899774,100033035,2419168,2419168,1,0,0
5535986,12899775,101743151,2419170,2419170,1,0,0
5535987,12899776,100548599,2419172,2419172,1,0,0
5535988,12899777,100384045,2419176,2419176,1,0,0


In [9]:
import numpy as np
edges_val = edges.values

In [10]:
import numpy as np
edges_test_val = edges_test.values

In [11]:
all_edges = np.vstack([edges_val, edges_test_val])

In [12]:
del edges_val, edges_test_val, edges, edges_test

In [13]:
all_edges.shape

(99818502, 7)

In [14]:
PR_CLICK_WEIGHT = 1
PR_CART_WEIGHT = 3
PR_ORDER_WEIGHT = 6
weight = all_edges[:, -3] * PR_CLICK_WEIGHT + all_edges[:, -2] * PR_CART_WEIGHT + all_edges[:, -1] * PR_ORDER_WEIGHT 

In [15]:
weight = weight.reshape(-1, 1)

In [16]:
all_edges = np.hstack([all_edges, weight])
all_edges.shape

(99818502, 8)

In [17]:
del weight

# Wait to delete

In [18]:
elts = ug.new_ep('unsigned long')
efts = ug.new_ep('unsigned long')
eclick = ug.new_ep('unsigned int')
ecart = ug.new_ep('unsigned int')
eorder = ug.new_ep('unsigned int')
eweight = ug.new_ep('unsigned int')


In [19]:
vmap = ug.add_edge_list(all_edges, eprops=[elts, efts, eclick, ecart, eorder, eweight])

In [20]:
ug.num_edges()

99818502

In [21]:
del all_edges

# Pagerank

In [1300]:
#get pagerank
import graph_tool.centrality as gt
pr = gt.pagerank(ug, weight = eweight)

In [1301]:
#get log_pagerank
from graph_tool import map_property_values
log_pr = ug.new_vertex_property("double")
map_property_values(pr, log_pr,
                       lambda x: 10 + np.log10(x))

# Load test

In [85]:
# ROOT = '/content/drive/MyDrive/ColabNotebooks/điệp viên'
# DATA_FOLDER = ROOT + '/data'



In [86]:
ROOT = '/content/drive/MyDrive/ColabNotebooks/điệp viên'
DATA_FOLDER = ROOT + '/splitted_data'



In [87]:
import pandas as pd
test_df = pd.read_parquet(DATA_FOLDER + '/test.parquet')

In [88]:
test_df['aid'] = test_df['aid'] + 100000000

In [89]:
TRAIN_TS_MIN = 1659304800
test_df['ts'] = test_df['ts'] - TRAIN_TS_MIN

In [90]:
test_df = test_df.loc[:,['session', 'ts']].groupby('session').agg({'ts': ['last', 'first']}).reset_index()

In [91]:
test_df = test_df.iloc[: test_df.shape[0] // 100]

In [92]:
test_df

session       ts         
                    last    first
0      11098528  1814400  1814400
1      11098529  1814400  1814400
2      11098530  1815732  1814400
3      11098531  1814946  1814400
4      11098532  1815196  1814401
...         ...      ...      ...
18007  11116535  2102349  1839150
18008  11116536  1839150  1839150
18009  11116537  1839393  1839154
18010  11116538  1839155  1839155
18011  11116539  1839155  1839155

[18012 rows x 3 columns]

#Find neighbour strategy


In [1326]:
import numba as nb
DAY = 3600 * 24

@nb.njit(nb.float64[:](nb.int64[:]))
def normalize(X):
  m = np.max(X)
  return (m - X) / (m - np.min(X) + 1e-12) 

@nb.njit([nb.float64[:](nb.float64[:], nb.float64, nb.float64),
          nb.float64[:](nb.float64[:], nb.int64, nb.float64),
          nb.float64[:](nb.float64[:], nb.float64, nb.int64),
         nb.float64[:](nb.float64[:], nb.int64, nb.int64)])
def normalize_to_range(X, l, r):
  m = np.min(X)
  return (X - m) / (np.max(X) - m + 1e-12) * (r -l) + l



@nb.njit([nb.types.UniTuple(nb.float64[:],2) (nb.float64[:], nb.float64[:]),
          nb.types.Tuple((nb.int64[:], nb.float64[:])) (nb.int64[:], nb.float64[:]),
          ])
def sum_by_group(groups, values):
    order = np.argsort(groups)
    groups = groups[order]
    values = values[order]
    values = np.cumsum(values)
    index = np.ones(len(groups), 'bool')
    index[:-1] = groups[1:] != groups[:-1]
    values = values[index]
    groups = groups[index]
    values[1:] = values[1:] - values[:-1]
    return groups, values


@nb.njit([nb.types.UniTuple(nb.float64[:],4) (nb.float64[:], nb.float64[:], nb.float64[:]),
          nb.types.Tuple((nb.int64[:], nb.float64[:], nb.int64[:], nb.float64[:])) (nb.int64[:], nb.float64[:], nb.int64[:]),
          ])
def sum_by_group_exclude(groups, values, exclude):

    groups, values = sum_by_group(groups, values)

    order = np.argsort(-values)[:40]
    groups = groups[order]
    values = values[order]

    inclusive = np.ones(len(groups), 'bool')
    for e in exclude:
      inclusive = inclusive & (groups != e)
    
    return groups[inclusive], values[inclusive], groups[~inclusive], values[~inclusive]


def get_item_neighbours(user, lts, fts, alpha = 0.5, use_pagerank = False, use_log = True):
  #edge idx: 0 is src, 1 is target, 2 is last time stamp, 3 is first time stamp, 4 click, 5 cart, 6 order
  edges = ug.get_all_edges(user, eprops = [elts, efts, eclick, ecart, eorder])
  pagerank = ug.get_all_neighbors(user, vprops = [log_pr if use_log else pr])[:, 1] if use_pagerank else 1
  return _get_item_neighbours(edges, lts, fts, pagerank, alpha)

@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:]))(nb.int64[:,:],nb.int64, nb.int64, nb.float64[:], nb.float64), 
          nb.types.Tuple((nb.int64[:], nb.float64[:]))(nb.int64[:,:],nb.int64, nb.int64, nb.int64, nb.float64), 
          ])
def _get_item_neighbours(edges, lts, fts, pagerank, alpha):
  #edge idx: 0 is src, 1 is target, 2 is last time stamp, 3 is first time stamp, 4 click, 5 cart, 6 order


  weight = edges[:, 4] * CLICK_WEIGHT + edges[:, 5] * CART_WEIGHT + edges[:, 6] * ORDER_WEIGHT


  #check if union, if yes time diff = 0, else = min(fts1 - lts2, fts2 - lts1)
  is_end_later = np.where(edges[:, 2] > lts, True, False)

  is_union = np.empty(edges.shape[0], dtype = np.bool_)
  is_union[is_end_later] = np.where(edges[is_end_later, 3] < lts, True, False)
  is_end_sooner = ~is_end_later
  is_union[is_end_sooner] = np.where(edges[is_end_sooner, 2] > fts, True, False)

  time_diff = np.empty(edges.shape[0],  dtype = np.float32)
  
  time_diff[is_union] = 0

  not_union_and_later = ~is_union & is_end_later
  not_union_and_sooner = ~is_union & is_end_sooner

  time_diff[not_union_and_later] = np.abs(lts - edges[not_union_and_later, 3]) / DAY
  time_diff[not_union_and_sooner] = np.abs(fts - edges[not_union_and_sooner, 2]) / DAY
  time_decay = alpha ** time_diff
  
  fitness = weight * time_decay * pagerank
  return sum_by_group(edges[:, 1], fitness)


def get_item_neighbours_byts(user, beta = 0.8):
  #0: src, 1: target, 2: lts, 3: eclick, 4: ecart, 5:eorder
  edges = ug.get_all_edges(user, eprops = [elts, eclick, ecart, eorder])

  
  return _get_item_neighbours_byts(edges, beta)

@nb.njit(nb.types.Tuple((nb.int64[:], nb.float64[:]))(nb.int64[:, :], nb.float64) )
def _get_item_neighbours_byts(edges, beta = 0.8):
  #0: src, 1: target, 2: lts, 3: eclick, 4: ecart, 5:eorder
  
  time_decay = beta ** normalize(edges[:, 2])

  order_weight = np.where(edges[:, 5] > 1, TEST_ORDER_WEIGHT_L, TEST_ORDER_WEIGHT_S)

  weight = edges[:, 3] * TEST_CLICK_WEIGHT + edges[:, 4] * TEST_CART_WEIGHT + order_weight 

  fitness =  time_decay * weight

  order = np.argsort(- fitness )  

  return edges[order, 1], fitness +  order_weight / 2
  



def get_user_neighbours(item):
  edges = ug.get_all_edges(item, eprops = [elts, efts])
  return _get_user_neighbours(edges)

@nb.njit(nb.types.UniTuple(nb.int64[:],3)(nb.int64[:, :]))
def _get_user_neighbours(edges):
  return edges[:,1], edges[:,2], edges[:,3]

@nb.njit(nb.int64[:](nb.int64[:], nb.float64[:], nb.int64[:], nb.float64[:]))
def match_and_rerank(items, fitness, ori_items, ori_fitness):
  #calculate covisit from known items and rerank

  #ori_items already sorted ascending  
  order = np.argsort(items)


  ori_fitness = normalize_to_range(ori_fitness, RERANK_L, RERANK_R)
  
  j = 0
  n = len(order)
  print('hello')
  # every elements of ori is in items
  for o in order:
    if items[o] < ori_items[j]:
      continue
    
    #enter here means equal
    fitness[o] *= ori_fitness[j]
    j+=1
  
  order = np.argsort(- fitness)
  return items[order]

@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:], nb.int64[:], nb.boolean[:])) (nb.int64[:], nb.float64[:]),
          nb.types.Tuple((nb.int64[:], nb.int64[:], nb.int64[:], nb.boolean[:])) (nb.int64[:], nb.int64[:]),
          ])
def find_topk_user(cousers, cofitness):
  #sort cousers ascending by id
  order = np.argsort(cousers)
  cousers = cousers[order]
  cofitness = cofitness[order]

  #calculate groupby sum of cousers
  cumcofitness = np.cumsum(cofitness)

  #find unique
  index = np.ones(len(cousers), 'bool')

  #this is the last index of each group
  index[:-1] = cousers[1:] != cousers[:-1]
  cumcofitness = cumcofitness[index]
  

  #find start and end of groups
  nodes = np.hstack((np.array([0]), np.where(index)[0] + 1))
    
  #caluculate groupby sum of cousers
  cumcofitness[1:] = cumcofitness[1:] - cumcofitness[:-1]

  #get topk user
  order_fit = np.argsort(-cumcofitness)[:TOPK_COUSER]

  starts = nodes[order_fit]
  ends = nodes[order_fit + 1]
  
  filter = np.zeros(len(cousers), 'bool')
  for i in range(len(starts)):
    filter[starts[i] : ends[i]] = 1
  
  return cousers[filter], cofitness[filter], order, filter



In [1327]:
import time

def step(user, lts, fts, preds, lens):
  #get item in tests

  items, fitness = get_item_neighbours_byts(user, BETA)
  
  lens.append(len(items))

  
  num_more = 20 - len(items)
  
  
  
  #get couser
  coalls = [get_user_neighbours(items[i]) for i in range(len(items))]

  cofitness = np.hstack([np.full_like(coalls[i][0], fitness[i]) for i in range(len(items))])
  

  #find topk cousers
  cousers, cofitness, order, filter  = find_topk_user(np.hstack([all[0] for all in coalls]), cofitness)
  coltss = np.hstack([all[1] for all in coalls])[order][filter]
  coftss = np.hstack([all[2] for all in coalls])[order][filter]


  #get coitems
  
  coalls = [get_item_neighbours(cousers[i], coltss[i], coftss[i], ALPHA, use_pagerank = USE_PAGERANK, use_log = USE_LOG) for i in range(len(cousers))]

  cofitness = np.hstack([all[1] * cofitness[i] for i, all in enumerate(coalls)]) 
  coitems = np.hstack([all[0] for all in coalls])
  

  
  #count covisit

  coitems, cofitness, ori_items, ori_fitness = sum_by_group_exclude(coitems, cofitness, exclude =  items)
  

  #match and rerank ori_item

  if len(items)  > 19:
    # items = match_and_rerank(items, fitness, ori_items, ori_fitness)
    preds.append(items.tolist()[:20])

  else:    
    #sort by count then to fitness
    
    preds.append(items.tolist() +  coitems[:num_more].tolist())
    
  
    
  


# Single Processing

In [928]:
test_dict = test_df.to_dict('records')



In [929]:
from tqdm.notebook import tqdm
import time

preds = []
lens = []
k = 1
TOPK_COUSER = 100

func_list = ['get_item_in_test', 'get_couser', 'count_couser', 'get_coitems', 'count_covisit', 'rank_covisit', 'couser_unique', 'total']
time_dict = dict(zip(func_list, [[] for _ in range(len(func_list))]))

len_co = []

for row in tqdm(test_dict):
  k += 1

  total_t = time.time()


  user, lts, fts = row['session', ''], row['ts', 'last'], row['ts', 'first']
  
  step(user, lts, fts, preds, lens)
  
  # if k == 100:
  #   break






  0%|          | 0/18012 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
preds

In [ ]:
np.mean(len_co)

In [ ]:
time_summary = ''
total = np.sum(time_dict['total'])
for func in func_list:
    time_summary += f'Get {func} : {np.sum(time_dict[func])}, {100 * np.sum(time_dict[func]) / total} % \n'
print(time_summary)
# ,np.mean(get_couser,),np.mean(get_coitems),np.mean(count_covisit),np.mean(rank_covisit )

# Multiprocessing

In [1328]:
from tqdm import tqdm
import multiprocessing

def batch_proc(proc_num, batch, return_dict):
  preds = []
  lens = []
  for  row in tqdm(batch):

    
    user, lts, fts = row['session', ''], row['ts', 'last'], row['ts', 'first']
    
    step(user, lts, fts, preds, lens)

  return_dict[proc_num] =  preds, lens
    

In [1329]:
test_dict = test_df.to_dict('records')



In [1330]:
#divide into batches
NUM_CPU = multiprocessing.cpu_count()
batch_size = len(test_dict) // NUM_CPU
start = np.arange(NUM_CPU) * batch_size
end = start + batch_size
end[-1] = len(test_dict)
batches = [test_dict[start[i] : end[i]] for i in range(len(start))]

In [1331]:
manager = multiprocessing.Manager()
return_dict = manager.dict()
jobs = [multiprocessing.Process(target=batch_proc, args=(i, batch, return_dict)) for i, batch in enumerate(batches)]
len(jobs)

40

In [1332]:
%%time
for job in jobs:
  job.start()
for job in jobs:
  job.join()
# return_dict[0]

100%|██████████| 450/450 [00:22<00:00, 20.35it/s]


CPU times: user 10.6 s, sys: 35.5 s, total: 46.1 s
Wall time: 27.7 s


In [1333]:
preds = []
lens = []
from tqdm.notebook import tqdm
for i in tqdm(range(len(return_dict))):
  preds.extend(return_dict[i][0])
  lens.extend(return_dict[i][1])

  0%|          | 0/40 [00:00<?, ?it/s]

# Create submission

In [1334]:
preds[0]

[100011830,
 100588923,
 101157882,
 100884502,
 100231487,
 101182614,
 100307904,
 100756588,
 100876129,
 100571762,
 101790438,
 101732105,
 101426451,
 100577040,
 100045494,
 100205357,
 101683666,
 101689044,
 100943735,
 101517680]

In [1335]:
count = 0
missing = []
for i, p in enumerate(preds):
  if len(p) != 20:
    missing.append(i)

count

0

In [1336]:
# new_append = [' '.join([str(int(i) - 100000000) for i in a]) for a in preds]

In [1337]:
new_append = [[int(j) - 100000000 for j in i] for i in preds]

In [1338]:
submission1 = test_df.copy()
submission1['labels'] = new_append
submission2 = submission1.copy()
submission3 = submission1.copy()

In [1339]:
submission1

session       ts           \
                    last    first   
0      11098528  1814400  1814400   
1      11098529  1814400  1814400   
2      11098530  1815732  1814400   
3      11098531  1814946  1814400   
4      11098532  1815196  1814401   
...         ...      ...      ...   
18007  11116535  2102349  1839150   
18008  11116536  1839150  1839150   
18009  11116537  1839393  1839154   
18010  11116538  1839155  1839155   
18011  11116539  1839155  1839155   

                                                  labels  
                                                          
0      [11830, 588923, 1157882, 884502, 231487, 11826...  
1      [1105029, 1592351, 323714, 1571965, 1072035, 1...  
2      [409236, 264500, 1603001, 364155, 254154, 8774...  
3      [1271998, 1728212, 1365569, 396199, 452188, 15...  
4      [876469, 7651, 108125, 1202618, 612920, 164599...  
...                                                  ...  
18007  [1573399, 1851286, 1263599, 252993, 1136979, 1...  
18008  [560048, 485256, 926412, 1734475, 1005087, 803...  
18009  [5704, 128446, 409513, 1096852, 1316140, 70603...  
18010  [1778843, 836852, 852331, 825591, 835958, 4354...  
18011  [68703, 1196256, 441688, 688503, 1368794, 1816...  

[18012 rows x 4 columns]

In [1340]:
submission1['session_type'] = submission1['session'].apply(lambda x: str(x) + '_clicks')
submission2['session_type'] = submission1['session'].apply(lambda x: str(x) + '_carts')
submission3['session_type'] = submission1['session'].apply(lambda x: str(x) + '_orders')

In [1341]:
final = pd.concat([submission1, submission2, submission3])

In [1342]:
del submission1, submission2, submission3

In [1343]:
del final['session'], final['ts']


In [1344]:
final = final.droplevel(1, axis = 1)

In [1345]:
final

,labels,session_type
0,"[11830, 588923, 1157882, 884502, 231487, 11826...",11098528_clicks
1,"[1105029, 1592351, 323714, 1571965, 1072035, 1...",11098529_clicks
2,"[409236, 264500, 1603001, 364155, 254154, 8774...",11098530_clicks
3,"[1271998, 1728212, 1365569, 396199, 452188, 15...",11098531_clicks
4,"[876469, 7651, 108125, 1202618, 612920, 164599...",11098532_clicks
...,...,...
18007,"[1573399, 1851286, 1263599, 252993, 1136979, 1...",11116535_orders
18008,"[560048, 485256, 926412, 1734475, 1005087, 803...",11116536_orders
18009,"[5704, 128446, 409513, 1096852, 1316140, 70603...",11116537_orders
18010,"[1778843, 836852, 852331, 825591, 835958, 4354...",11116538_orders


# Eval valid

In [1323]:
import pandas as pd

def get_score(df, split = False):
    submission = df.copy()
    test_labels = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/điệp viên/splitted_data/test_labels.parquet')
    test_labels = test_labels[test_labels['session'] <= test_df['session'].max()]
    submission['session'] = submission.session_type.apply(lambda x: int(x.split('_')[0]))
    submission['type'] = submission.session_type.apply(lambda x: x.split('_')[1])
    
    if split:
      submission.labels = submission.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])

    
    test_labels = test_labels.merge(submission, how='left', on=['session', 'type'])
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)

    del submission

    recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum() 

    print (f"Score : {(recall_per_type * pd.Series({'clicks': 0.10, 'carts': 0.30, 'orders': 0.60})).sum()}")
    return recall_per_type

In [1346]:
get_score(final)

Score : 0.4962978362803555


type
carts     0.343268
clicks    0.448837
orders    0.580723
dtype: float64

# Save sumission

In [1299]:
final.to_csv('/content/drive/MyDrive/ColabNotebooks/12_8_4_submission.csv', index = False)

#Submit


In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%bash
rm ~/.kaggle/kaggle.json
mkdir ~/.kaggle
cp /content/drive/MyDrive/ColabNotebooks/kaggle.json ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle competitions submit -c otto-recommender-system -f /content/drive/MyDrive/ColabNotebooks/12_8_4_submission.csv -m "563 + rerank "

100% 782M/782M [00:14<00:00, 55.7MB/s]
Successfully submitted to OTTO – Multi-Objective Recommender System

#Submit 1

In [ ]:
!pip install kaggle

In [ ]:
%%bash
rm ~/.kaggle/kaggle.json
mkdir ~/.kaggle
cp /content/drive/MyDrive/ColabNotebooks/kaggle1.json ~/.kaggle
mv ~/.kaggle/kaggle1.json ~/.kaggle/kaggle.json
chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle competitions submit -c otto-recommender-system -f /content/drive/MyDrive/ColabNotebooks/12_8_5_submission.csv -m "without train rerank revalid"

# Create submission without add train

In [1219]:
preds[0]

[100011830,
 100588923,
 101157882,
 100884502,
 100231487,
 100307904,
 101182614,
 100876129,
 101683666,
 100756588,
 100571762,
 101732105,
 101426451,
 100577040,
 101790438,
 100205357,
 100045494,
 101448179,
 101657173,
 100943735]

In [1220]:
count = 0
missing = []
for i, p in enumerate(preds):
  if len(p) != 20:
    missing.append(i)

count

0

In [1221]:
# new_append = [' '.join([str(int(j) - 100000000) for j in i[ : lens[m]]]) for m, i in enumerate(preds)]

In [1222]:
new_append = [[int(j) - 100000000 for j in i[ : lens[m]]] for m, i in enumerate(preds)]

In [1223]:
submission1 = test_df.copy()
submission1['labels'] = new_append
submission2 = submission1.copy()
submission3 = submission1.copy()

In [1224]:
submission1

session       ts           \
                    last    first   
0      11098528  1814400  1814400   
1      11098529  1814400  1814400   
2      11098530  1815732  1814400   
3      11098531  1814946  1814400   
4      11098532  1815196  1814401   
...         ...      ...      ...   
18007  11116535  2102349  1839150   
18008  11116536  1839150  1839150   
18009  11116537  1839393  1839154   
18010  11116538  1839155  1839155   
18011  11116539  1839155  1839155   

                                                  labels  
                                                          
0                                                [11830]  
1                                              [1105029]  
2                                       [409236, 264500]  
3      [1271998, 1365569, 1728212, 396199, 452188, 15...  
4                                         [7651, 876469]  
...                                                  ...  
18007                [1573399, 252993, 1263599, 1851286]  
18008                                           [560048]  
18009                             [128446, 5704, 409513]  
18010                                          [1778843]  
18011                                            [68703]  

[18012 rows x 4 columns]

In [1225]:
submission1['session_type'] = submission1['session'].apply(lambda x: str(x) + '_clicks')
submission2['session_type'] = submission1['session'].apply(lambda x: str(x) + '_carts')
submission3['session_type'] = submission1['session'].apply(lambda x: str(x) + '_orders')

In [1226]:
final = pd.concat([submission1, submission2, submission3])

In [1227]:
del submission1, submission2, submission3

In [1228]:
del final['session'], final['ts']


In [1229]:
final = final.droplevel(1, axis = 1)

In [1230]:
final

,labels,session_type
0,[11830],11098528_clicks
1,[1105029],11098529_clicks
2,"[409236, 264500]",11098530_clicks
3,"[1271998, 1365569, 1728212, 396199, 452188, 15...",11098531_clicks
4,"[7651, 876469]",11098532_clicks
...,...,...
18007,"[1573399, 252993, 1263599, 1851286]",11116535_orders
18008,[560048],11116536_orders
18009,"[128446, 5704, 409513]",11116537_orders
18010,[1778843],11116538_orders


# Save sumission

In [ ]:
final.to_csv('/content/drive/MyDrive/ColabNotebooks/12_8_5_submission.csv', index = False)